In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import math
import nolds
from PreprocessFcns import *
%matplotlib inline

In [2]:
full_dict = {}
count = 0
subj_list = ([1004,1016,1018,1019,1020,1024,1029,1030,1032,1038,1044,1046,1047,1049,1051,1052,1053,1054,1055,1056])
for subj in subj_list:
    f = open('Z:\CIS-PD Study\Data_dict\\' + str(subj) + 'dict.pkl', 'rb')
    full_dict[subj] = (pickle.load(f))
    f.close()
    count = count + 1
    if count < 10:
        print(' ' + str(count) + ' of 20 subjects loaded')
    else:
        print(str(count) + ' of 20 subjects loaded')

 1 of 20 subjects loaded
 2 of 20 subjects loaded
 3 of 20 subjects loaded
 4 of 20 subjects loaded
 5 of 20 subjects loaded
 6 of 20 subjects loaded
 7 of 20 subjects loaded
 8 of 20 subjects loaded
 9 of 20 subjects loaded
10 of 20 subjects loaded
11 of 20 subjects loaded
12 of 20 subjects loaded
13 of 20 subjects loaded
14 of 20 subjects loaded
15 of 20 subjects loaded
16 of 20 subjects loaded
17 of 20 subjects loaded
18 of 20 subjects loaded
19 of 20 subjects loaded
20 of 20 subjects loaded


In [4]:
#save corrected version of motor scores.xls 
mot_scores = pd.read_excel('Z:CIS-PD Study\Scores\MotorTasks.xls')
#fix error in original cols labels
mot_scores.columns.values[75] = 'alternating left hand movements tremor right upper limb'
mot_scores.columns.values[150] = 'sitting tremor right upper limb'
#remove words:(Qxx) and 'rating' from each column for readability
cols= mot_scores.columns
cols = cols.tolist()
colsnew = [x.split('(')[0] for x in cols]
colsnew = [x.strip() for x in colsnew] #remove whitespace
colsnew = [x.split('rating')[0] for x in colsnew]
colsnew = [x.strip() for x in colsnew]
colsnew = [x.lower() for x in colsnew] #make all lower case 
colsnew = [x.replace('\x97',' ') for x in colsnew]
colsnew = [x.replace('—',' ') for x in colsnew]    
#simplify notation
for i in range(len(colsnew)):
    x = colsnew[i]
    if x.find('finger to nose')>-1:
        colsnew[i] = x.replace(' hand','')
    if x.find('alternating')>-1:
        colsnew[i] = x.replace(' hand movements','')
c = dict(zip(cols,colsnew))
mot_scores = mot_scores.rename(index=str, columns=c)
#change 1 month label to 4 wks for proper sorting (to incorporate in xls file)
mot_scores.loc[mot_scores['visit']=='1 Month','visit']='4 Weeks'
mot_scores.to_excel('Z:CIS-PD Study\Scores\MotorTasks.xls') #note that timestamps are not properly converted

In [36]:
def DataAggregator(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=27.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 27.5Hz.csv')

In [37]:
subj_list = [1004,1016,1018,1019,1020,1024,1029,1030,1032,1038,1044,1046,1047,1049,1051,1052,1053,1054,1055,1056]
task_list = ['Motor #1: Standing','Motor #2: Walking', 'Motor #3: Walking while Counting','Motor #4: Finger to Nose',
             'Motor #5: Alternating Hand Movements','Motor #6: Sit to Stand','Motor #7: Drawing on Paper',
             'Motor #8: Typing on a Computer','Motor #9: Nuts and Bolts','Motor #10: Drinking Water',
             'Motor #11: Organizing Folder','Motor #12: Folding Towels','Motor #13: Sitting']
task_scores = ['standing','walking','walking while counting','finger to nose','alternating','sit to stand',
               'drawing on a paper','typing on a computer keyboard','assembling nuts and bolts',
               'taking a glass of water and drinking','organizing sheets in a folder','folding towels','sitting']
side_list = ['left','right']
session_list = ['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', '2 Weeks: Time 120', 
               '2 Weeks: Time 150', '4 Weeks']

In [38]:
DataAggregator(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)


In [39]:
def DataAggregator225(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=22.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 22.5Hz.csv')

In [40]:
DataAggregator225(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)


In [41]:
def DataAggregator175(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=17.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 17.5Hz.csv')

In [42]:
DataAggregator175(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)


In [43]:
def DataAggregator125(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=12.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 12.5Hz.csv')

In [44]:
DataAggregator125(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)


In [45]:
def DataAggregator075(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=7.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 7.5Hz.csv')

In [46]:
DataAggregator075(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)


In [47]:
def DataAggregator025(full_dict,subj_list,task_list,task_scores,side_list,session_list):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in subj_list:
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_list)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            for side in side_list:
                
                loc = 'dorsal_hand_' + side
                
                if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                    task_score = ts + ' ' + side
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                
                n_visits = len(subj_score)
                n_recordings = len(act_dict[task].keys())
                N = n_visits

                if n_visits != n_recordings:
                    #print('Recordings error')
                    N = min([n_visits,n_recordings])

                for trial in range(N):

                    if len(act_dict[task][trial][loc][sensor]) <= 6:
                        continue

                    ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                    act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                    data = HPfilter(act_dict[task][trial][loc][sensor])

                    clip_data = gen_clips(data,downsample=2.5)
                    feature_extraction(clip_data)

                    if 'features' in clip_data.keys():
                        D = clip_data['features']
                        featcols = list(D.columns)
                        D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                        D['Tremor'] = subj_score['Tremor'][trial]
                        D['Visit'] = session_list[trial]
                        D['Task'] = task
                        D['Subject'] = subj
                        Data = pd.concat([Data,D])
                            
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Accel Data 2.5Hz.csv')

In [48]:
DataAggregator025(full_dict,subj_list,task_list,task_scores,side_list,session_list)

Subject 1004 (1 of 20)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 20)
Subject 1018 (3 of 20)
Subject 1019 (4 of 20)
Subject 1020 (5 of 20)
Subject 1024 (6 of 20)
Subject 1029 (7 of 20)
Subject 1030 (8 of 20)
Subject 1032 (9 of 20)
Subject 1038 (10 of 20)
Subject 1044 (11 of 20)
Subject 1046 (12 of 20)
Subject 1047 (13 of 20)
Subject 1049 (14 of 20)
Subject 1051 (15 of 20)
Subject 1052 (16 of 20)
Subject 1053 (17 of 20)
Subject 1054 (18 of 20)
Subject 1055 (19 of 20)
Subject 1056 (20 of 20)
